In [87]:
import pandas as pd
import numpy as np 
from matplotlib import pyplot as plt 
from IPython.display import display
from tqdm import tqdm_notebook
import tqdm
import seaborn as sns
from choices import param_dict
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import  IterativeImputer 
from sklearn.impute import SimpleImputer , KNNImputer
from sklearn.preprocessing import Normalizer
from sklearn.ensemble import GradientBoostingClassifier
sns.set(font_scale=1.3, rc={'axes.facecolor':'white', 'figure.facecolor':'white' , 'axes.grid':True} , style="whitegrid")
# 
%load_ext autoreload
%autoreload 2
import time 
tstart = time.time()
feat_to_drop = param_dict['hardness']+param_dict['IRAC']

In [88]:

from utilities import deets
from choices import get_train_data , param_dict
classes = ['AGN' ,'STAR' , 'YSO' ,  'CV' , 'LMXB' , 'HMXB' ,'ULX','PULSAR']
flag = {
    'conf_flag' : 0 , 
    'streak_src_flag' : 0 , 
    'extent_flag' : 0 , 
    'pileup_flag' : 0 , 
    }
ret_dict= {
    'clf': False,
 'prob_table': True,
 'acc': True,
 'pr_score': True,
 'precision': True,
 'recall': True , 
 'roc_auc_score' : False
 }



In [89]:

file = f'../compiled_data_v3/imputed_data_v2/x_phot_minmax_modeimp.csv'
data = get_train_data(flags = flag, classes= classes , offset = 1, file=file)
data = data.drop(columns = feat_to_drop)
deets(data,0)


_____________________________________________________
------------------------------
Number of Objects : 7703
Number of Columns : 42
_____________________________________________________


In [90]:
import joblib
from tqdm import tqdm
from utilities import cv , get_score
from sklearn.ensemble import GradientBoostingClassifier , RandomForestClassifier

# {'max_depth': 30, 'n_estimators': 400} for MODE imputation
# {'max_depth': 180, 'n_estimators': 400} for RF imputation
for i in (range(5)):
    print('-------------------------------------')
    print(f'DOING {i}th Iteration')
    rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1 , random_state=np.random.randint(0,999999))
    res_final  = cv(
        {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
        {'model' : rf , 'name' :'GB'} , ## CHANGE HERE########################################################
        k = 20 , return_dict = ret_dict, save_df= False , multiprocessing = False)
    score = get_score([res_final['prob_table']])
    joblib.dump(score , f'result/classifier_scores/rf_mode_{i}.pkl')
    joblib.dump(score , f'result/prob_table/rf_mode_{i}.pkl' , compress=3)

-------------------------------------
DOING 0th Iteration
Doing 20 fold cross-validation


100%|██████████| 20/20 [02:06<00:00,  6.32s/it]


[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 1th Iteration
Doing 20 fold cross-validation


100%|██████████| 20/20 [02:06<00:00,  6.33s/it]


[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 2th Iteration
Doing 20 fold cross-validation


...

[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 3th Iteration
Doing 20 fold cross-validation


100%|██████████| 20/20 [02:02<00:00,  6.10s/it]


[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 4th Iteration
Doing 20 fold cross-validation


100%|██████████| 20/20 [02:05<00:00,  6.25s/it]


[RESULT] Overall Accuracy : 0.91
[INFO] Validation probability table is available as ['prob_table'] in response


In [100]:
import joblib
from tqdm import tqdm
from utilities import cv , get_score
from sklearn.ensemble import GradientBoostingClassifier , RandomForestClassifier

# {'max_depth': 30, 'n_estimators': 400} for MODE imputation
# {'max_depth': 180, 'n_estimators': 400} for RF imputation
for i in (range(5)):
    print('-------------------------------------')
    print(f'DOING {i}th Iteration')
    # rf = RandomForestClassifier(n_estimators=400 , max_depth=30 , n_jobs=-1 , random_state=np.random.randint(0,999999))
    gb = GradientBoostingClassifier(random_state=np.random.randint(0,999999))
    res_final  = cv(
        {'data' : data , 'name' : f'data_10iter_rfimp_tuned'},   
        {'model' : gb, 'name' :'GB'} , ## CHANGE HERE########################################################
        k = 20 , return_dict = ret_dict, save_df= False , multiprocessing = True)
    score = get_score([res_final['prob_table']])
    joblib.dump(score , f'result/classifier_scores/gb_mode_{i}.pkl')
    joblib.dump(score , f'result/prob_table/gb_mode_{i}.pkl' , compress=3)

-------------------------------------
DOING 0th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.92
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 1th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.92
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 2th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.92
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 3th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.92
[INFO] Validation probability table is available as ['prob_table'] in response
-------------------------------------
DOING 4th Iteration
Doing 20 fold cross-validation
[RESULT] Overall Accuracy : 0.92
[INFO] Validation probability table is available as ['prob_table'] in resp

In [91]:
def score_mean(file):
    labels = ['AGN' ,'STAR' , 'YSO' , 'HMXB' , 'LMXB' , 'ULX' , 'CV' , 'PULSAR']
    scores = ['precision' , 'recall' , 'f1_score']
    all_feat = {
        'AGN' : {
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } ,
        'YSO' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'STAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'HMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'LMXB' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        } , 
        'ULX' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'CV':{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        },
        'PULSAR' :{
            'precision' : [] , 
            'recall' : [],
            'f1_score' : []
        }
    }
    for l in labels:
        for i in range(5):
            model = joblib.load(f'result/classifier_scores/{file}_{i}.pkl')
            all_feat[l]['precision'].append(model['class_scores']['precision_score'][l])
            all_feat[l]['recall'].append(model['class_scores']['recall_score'][l])
            all_feat[l]['f1_score'].append(model['class_scores']['f1_score'][l])
    all_mean = {}
    for l in labels:
        all_mean[l] = {}
        for s in scores:
            all_mean[l][s] = np.mean(all_feat[l][s])*100
    all_std = {}
    for l in labels:
        all_std[l] = {}
        for s in scores:
            all_std[l][s] = np.std(all_feat[l][s])*100
    mean_df = pd.DataFrame(all_mean)
    std_df = pd.DataFrame(all_std)
    return mean_df , std_df

In [97]:
score = score_mean('gb_mode')

In [98]:
score[0].T

,precision,recall,f1_score
AGN,95.264631,96.926931,96.088499
STAR,93.765180,95.189964,94.472156
YSO,90.668556,95.387293,92.967925
HMXB,89.124721,90.000000,89.558416
LMXB,92.513994,79.160839,85.311594
ULX,75.915068,59.336493,66.586785
CV,63.642046,48.915663,55.311027
PULSAR,38.565547,20.792079,26.987193


In [99]:
score[1].T

,precision,recall,f1_score
AGN,0.205249,0.104301,0.134160
STAR,0.159507,0.097238,0.115999
YSO,0.160266,0.258179,0.172223
HMXB,0.728563,0.196483,0.356804
LMXB,1.750823,0.279720,0.838017
ULX,1.342122,1.625254,0.982248
CV,1.205104,1.104235,1.060149
PULSAR,1.834673,0.885571,0.773063


In [72]:
score[1]

,AGN,STAR,YSO,HMXB,LMXB,ULX,CV,PULSAR
precision,0.153113,0.156210,0.210555,0.315574,0.448078,0.889318,1.970650,2.410174
recall,0.159759,0.017559,0.170548,0.181346,0.342586,0.975889,0.851936,1.154644
f1_score,0.073306,0.080487,0.088599,0.189966,0.300596,0.856258,1.065672,1.421311
